In [1]:
#ripple detection code 

In [ ]:
lfp_pos_df

In [ ]:
lfp_sampling_rate = (LFPBand & {'nwb_file_name' : nwb_file_name,
                               'filter_name' : 'Ripple 150-250 Hz'}).fetch('lfp_band_sampling_rate')

# Get animal speed upsampled to LFP sampling rate
lfp_pos_df = (IntervalPositionInfo & {'nwb_file_name' : nwb_file_name,
                                      'interval_list_name' : 'pos 0 valid times',
                                      'position_info_param_name' : 'default_lfp'}).fetch1_dataframe()

# Head speed in cm/s - because that is a param of the ripple detection detection 
head_speed = np.asarray(lfp_pos_df.head_speed)

# Position timestamps in s
pos_time = lfp_pos_df.index

# Get ripple-filtered LFP
lfp_ripple_object = (LFPBand & {'nwb_file_name' : nwb_file_name,
                                'filter_name' : 'Ripple 150-250 Hz'}).fetch_nwb()[0]

# Ripple-filtered LFP in AD units
# No need to convert to volts because ripple detection occurs on standardized data
lfp_ripple = np.asarray( lfp_ripple_object['filtered_data'].data, dtype='double' )

lfp_time = lfp_ripple_object['filtered_data'].timestamps[:]

lfp_interval_list = (IntervalList & {'nwb_file_name' : nwb_file_name,
                                     'interval_list_name' : 'lfp valid times'}).fetch1('valid_times')
position_interval_list = (IntervalList & {'nwb_file_name' : nwb_file_name,
                                          'interval_list_name' : 'pos 0 valid times'}).fetch1('valid_times')

overlap_interval_list = interval_list_intersect(lfp_interval_list, position_interval_list)

pos_ind, lfp_ind = find_overlapping_times([pos_time, lfp_time], overlap_interval_list)
##############
position_df = pd.DataFrame(pos_time[pos_ind],index =pos_time[pos_ind] )
position_df['speed']= head_speed[pos_ind]

# position_df.time
position_df

new_index = pd.Index(np.unique(np.concatenate(
                (position_df.time, lfp_time[lfp_ind]))), name='time1')

new_index

speed_df = (position_df.reindex(index=new_index)
               .interpolate(method='linear')
               .reindex(index=lfp_time[lfp_ind]))
speed_df
#############
lfp_electrode_ids = lfp_ripple_object['filtered_data'].electrodes[:].index
electrode_groups = [(Electrode() & {'nwb_file_name' : nwb_file_name,
                                    'electrode_id' : lfp_electrode}).fetch1('electrode_group_name')
                     for lfp_electrode in lfp_electrode_ids]

tetrode_ind = np.asarray( [int(group_name) <= 23 for group_name in electrode_groups] )
tetrode_ind[[4, 5, 17]] = False
tetrode_ind


ripple_data_ind = np.ix_(lfp_ind, tetrode_ind)
ripple_data_ind

ripple_times_df = Kay_ripple_detector(lfp_time[lfp_ind],
                                      lfp_ripple[ripple_data_ind],
                                      np.array(speed_df.speed.to_list()),
                                      lfp_sampling_rate,
                                      speed_threshold=4.0,
                                      minimum_duration=0.015,
                                      zscore_threshold=2.0,
                                      smoothing_sigma=0.004,
                                      close_ripple_threshold=0.0)